https://www.tensorflow.org/responsible_ai/privacy/tutorials/privacy_report

https://github.com/tensorflow/privacy/tree/master/tensorflow_privacy/privacy/privacy_tests/membership_inference_attack

Based Model is trained with regularization of the entropy of the softmax output vector

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tensorflow import keras
import tensorflow as tf

# Define the neural network architecture
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.reshape=nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.nonlinear = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.reshape(x)
        # x = torch.flatten(x, start_dim=1)
        # x = x.reshape(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x= self.nonlinear(x)
        # x = torch.softmax(x, dim=1)
        return x
    

# Custom loss function: CrossEntropyLoss with entropy regularization
class LowEntropic_CELoss(nn.Module):
    def __init__(self, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean', lambdy=0.1):
        super(LowEntropic_CELoss, self).__init__()
        self.cross_entropy = nn.CrossEntropyLoss(weight, size_average, ignore_index, reduce, reduction)
        self.lambdy = lambdy

    def forward(self, input, target):
        cross_entropy_loss = self.cross_entropy(input, target)
        softmax_probs = torch.softmax(input, dim=1)
        entropy_loss = -torch.sum(softmax_probs * torch.log(softmax_probs + 1e-9), dim=1).mean()
        return cross_entropy_loss + self.lambdy * entropy_loss



# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('data', train=False, transform=transform)

train_loader = DataLoader(train_data, batch_size=64*20, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64*20, shuffle=False)

lambdy=2.0

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MNISTClassifier().to(device)
criterion =  LowEntropic_CELoss(lambdy=lambdy)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 1
for epoch in range(epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()


In [3]:
model.to('cpu')

MNISTClassifier(
  (reshape): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (nonlinear): Softmax(dim=1)
)

In [4]:
import time
input_tensor=torch.randn((1, 1, 28, 28))

model.eval()
# Perform a warm-up run to avoid potential overhead caused by initial device setup
with torch.no_grad():
    _ = model(input_tensor)



num_iterations = 1000  # Choose a suitable number of iterations to average over

start_time = time.time()
with torch.no_grad():
    for _ in range(num_iterations):
        _ = model(input_tensor)
end_time = time.time()

inference_latency = (end_time - start_time) / num_iterations
#convert to microseconds
inference_latency = inference_latency * 1000000
print(f'Inference latency: {inference_latency:.20f} micro-seconds')


Inference latency: 86.46178245544433593750 micro-seconds
